# Import the required Libs

In [2]:
import pandas as pd
import requests

# Specify the wikipedia link

In [3]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
URL = requests.get(link).text

# Create the basic dataframe from the table

In [4]:
Database = pd.read_html(URL)
df = pd.DataFrame(Database[0])
df.columns=['Postal Code','Borough','Neighbourhood']
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Drop the 'Not assigned' Borough's and reset the index  

In [5]:
indexnames = df[df['Borough']=='Not assigned'].index
df = df.drop(indexnames)
df1=df.groupby("Postal Code").agg(lambda x:','.join(set(x)))
df1=df1.reset_index()
df1.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Display shape of the dataframe

In [6]:
(df1.shape)

(103, 3)

# Read the CSV file into a dataframe

In [7]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Join the two DF's on the Postal Code key

In [8]:
df3 = pd.concat([df1, df2], axis=1, join='inner')
df3.columns = ['PostalCode','Borough','Neighbourhood','Postal Code','Latitude','Longitude']
df3 = df3.drop('Postal Code', axis=1)
df3.columns = ['Postal Code','Borough','Neighbourhood','Latitude','Longitude'] 
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
df3.shape

(103, 5)

In [11]:
!pip install folium
import numpy as np
import folium
import matplotlib.pyplot as plt
!conda install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim

     |████████████████████████████████| 92kB 18.3MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0  

In [12]:
Toronto_lat=43.6532 #latitude value
Toronto_lng=-79.3832 #longitude value

# create map of Toronto using latitude and longitude values
map_toronto= folium.Map(location=[Toronto_lat,Toronto_lng],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df3['Latitude'],df3['Longitude'],
                                         df3['Borough'],df3['Neighbourhood']):
    label='{},{}'.format(neighbourhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)

map_toronto